In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
from skimage import io, transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import random
import math
import torch.nn.functional as F


In [2]:
classes=['0','1']
transform = transforms.Compose(
    [
     transforms.ToPILImage(),
     transforms.Resize((28,28)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5)),
    ])


In [13]:
class AgeDataSet(Dataset):
    def __init__(self,csv,root,transform=None):
        self.root=root
        self.data=pd.read_csv(csv)
        self.X=np.array(self.data.iloc[:,0]).reshape(-1,1)
        y=self.data.iloc[:,1]
        self.Y=np.array(y).reshape(-1,1)
        self.transform=transform

    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        print("X ",self.X)
        print("Calledddd!!!!!!!! ",idx,self.X[idx][0])
        path=self.root
        item=self.X[idx][0]
        item=path+item
        image = io.imread(item)
        label=self.Y[idx][0]
        #print(label)
        if self.transform:
            image = self.transform(image)


        return image,label

age=AgeDataSet('/work/test/Desktop/python/pytoch/Aero/dataset/train.csv','/work/test/Desktop/python/pytoch/Aero/dataset/training/train/',transform=transform)

train_size=int(0.8*len(age))
test_size=len(age)-train_size
train_dataset, test_dataset = torch.utils.data.dataset.random_split(age, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)


dataiter = iter(train_loader)
image,_ = dataiter.next()
print(image.shape)
image,_ = dataiter.next()
print(image.shape)


X  [['641.jpg']
 ['978.jpg']
 ['800.jpg']
 ...
 ['347.jpg']
 ['1239.jpg']
 ['1037.jpg']]
Calledddd!!!!!!!!  tensor(317) 593.jpg
torch.Size([1, 3, 28, 28])
X  [['641.jpg']
 ['978.jpg']
 ['800.jpg']
 ...
 ['347.jpg']
 ['1239.jpg']
 ['1037.jpg']]
Calledddd!!!!!!!!  tensor(708) 96.jpg
torch.Size([1, 3, 28, 28])


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size= 5, padding =2)
        self.fc1 = nn.Linear(14*14*20, 500)
        self.fc2 = nn.Linear(500, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 14*14*20)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model=CNN()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criteria=nn.CrossEntropyLoss()


In [5]:
import skimage
for epoch in range(5):
    ave_loss=0.0
    
    for i,data in enumerate(train_loader,0):
        inputs,labels =data
        #print(inputs.shape)
        inputs=torch.from_numpy(np.array(inputs))
        labels=torch.from_numpy(np.array(labels))
        optimizer.zero_grad()
        output=model(inputs)
        
        
        loss=criteria(output,labels)
        loss.backward()
        optimizer.step()
        ave_loss+=loss.item()
        if i%200==199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 100))
            ave_loss = 0.0
correct=0
total=0
with torch.no_grad():
    for inputs,labels in test_loader:
#         inputs=data['image']
#         labels=data['label']
        
        inputs=torch.from_numpy(np.array(inputs))
        labels=torch.from_numpy(np.array(labels))
        outputs=model(inputs)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

print('Accuracy of the network on the  test images: %0.6f %%' % (
    100 * correct / total))

Calledddd!!!!!!!!  tensor(594)
Calledddd!!!!!!!!  tensor(31)
Calledddd!!!!!!!!  tensor(596)
Calledddd!!!!!!!!  tensor(517)
Calledddd!!!!!!!!  tensor(749)
Calledddd!!!!!!!!  tensor(18)
Calledddd!!!!!!!!  tensor(746)
Calledddd!!!!!!!!  tensor(1266)
Calledddd!!!!!!!!  tensor(126)
Calledddd!!!!!!!!  tensor(407)
Calledddd!!!!!!!!  tensor(214)
Calledddd!!!!!!!!  tensor(64)
Calledddd!!!!!!!!  tensor(1043)
Calledddd!!!!!!!!  tensor(19)
Calledddd!!!!!!!!  tensor(889)
Calledddd!!!!!!!!  tensor(338)
Calledddd!!!!!!!!  tensor(334)
Calledddd!!!!!!!!  tensor(1167)
Calledddd!!!!!!!!  tensor(274)
Calledddd!!!!!!!!  tensor(664)
Calledddd!!!!!!!!  tensor(533)
Calledddd!!!!!!!!  tensor(49)
Calledddd!!!!!!!!  tensor(295)
Calledddd!!!!!!!!  tensor(545)
Calledddd!!!!!!!!  tensor(470)
Calledddd!!!!!!!!  tensor(595)
Calledddd!!!!!!!!  tensor(1176)
Calledddd!!!!!!!!  tensor(1155)
Calledddd!!!!!!!!  tensor(68)
Calledddd!!!!!!!!  tensor(843)
Calledddd!!!!!!!!  tensor(574)
Calledddd!!!!!!!!  tensor(148)
Calledddd

KeyboardInterrupt: 